In [9]:
# one off operation
!pip install -q duckdb

In [10]:
import duckdb
def duckdb_attach_lakehouse(workspace,LH):
    ########## get token using notebooks utils inside Fabric
    ########## or a service principals outside Fabric https://fivetran.com/docs/destinations/onelake/setup-guide
    try:
      token = notebookutils.credentials.getToken('storage')
      duckdb.sql(f""" CREATE or replace SECRET onelake ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}') """)
    except:
        import configparser
        config = configparser.ConfigParser()
        config.read("C:/KV/variable.ini")
        duckdb.sql(f""" CREATE or replace SECRET onelake_spn (  TYPE AZURE,  PROVIDER SERVICE_PRINCIPAL,
                        TENANT_ID '{config.get("myvars", "tenantId")}',
                        CLIENT_ID '{config.get("myvars", "appId")}',
                        CLIENT_SECRET '{config.get("myvars", "secret") }',
                        ACCOUNT_NAME 'onelake')
                     """)
    ########## check if the lakehouse has a schema 
    try:
        duckdb.sql(f""" SELECT  * FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/dbo/*") """).df()['file'].tolist()
        withschema = 'yes'
    except:
        withschema = 'no'
    sql_schema = set()
    sql_statements = set()
    if withschema == 'yes':
        list_tables = duckdb.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/*/_delta_log/*.json") """).df()['tables'].tolist()
        for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            schema = parts[-2]
            table = parts[-1]
            sql_schema.add(f"CREATE SCHEMA IF NOT EXISTS {schema};")
            sql_statements.add(f"""CREATE OR REPLACE VIEW {schema}.{table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{schema}/{table}');""")
        duckdb.sql(" ".join(sql_schema))
    else:
        list_tables = duckdb.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/_delta_log/*.json") """).df()['tables'].tolist()
        for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            table = parts[-1]
            sql_statements.add(f"""CREATE OR REPLACE VIEW {table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{table}');""")
    duckdb.sql(" ".join(sql_statements))
    duckdb.sql("SHOW ALL TABLES").show(max_width=100)

In [11]:
%%time
duckdb_attach_lakehouse("aemodev","storage")

┌──────────┬─────────┬─────────────┬──────────────────────┬────────────────────────────┬───────────┐
│ database │ schema  │    name     │     column_names     │        column_types        │ temporary │
│ varchar  │ varchar │   varchar   │      varchar[]       │         varchar[]          │  boolean  │
├──────────┼─────────┼─────────────┼──────────────────────┼────────────────────────────┼───────────┤
│ memory   │ main    │ calendar    │ [date, year, month]  │ [TIMESTAMP WITH TIME ZON…  │ false     │
│ memory   │ main    │ duid        │ [DUID, Region, Fue…  │ [VARCHAR, VARCHAR, VARCH…  │ false     │
│ memory   │ main    │ mstdatetime │ [SETTLEMENTDATE, t…  │ [TIMESTAMP WITH TIME ZON…  │ false     │
│ memory   │ main    │ price       │ [UNIT, VERSION, SE…  │ [VARCHAR, DOUBLE, TIMEST…  │ false     │
│ memory   │ main    │ scada       │ [UNIT, DUID, filen…  │ [VARCHAR, VARCHAR, VARCH…  │ false     │
│ memory   │ main    │ states      │ [RegionID, States]   │ [VARCHAR, VARCHAR]         │ fa